In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Get the data
data1 = np.loadtxt("HARPS.dat")
data2 = np.loadtxt("HIRES.dat")
data = np.vstack((data1,data2))
print len(data1),' HARPS points; ',len(data2),' HIRES points'

# Add on jitter (additional noise term)
jitter = 1.2
for i in range(len(data)):
    data[i,2] = np.sqrt( data[i,2]**2 + jitter**2 )

# Initial information from Vogt et al. (2010)
Pb_sol = 5.36841; Pb_del = 0.00026; Kb_sol = 12.45
Pc_sol = 12.9191; Pc_del = 0.0058;  Kc_sol = 3.30
Pd_sol = 66.87;   Pd_del = 0.13;    Kd_sol = 1.91
Pe_sol = 3.14867; Pe_del = 0.00039; Ke_sol = 1.66
Pf_sol = 433.0;   Pf_del = 13.0;    Kf_sol = 1.30
Pg_sol = 36.562;  Pg_del = 0.052;   Kg_sol = 1.29

119  HARPS points;  122  HIRES points


In [5]:
# A full fit is tricky because it's not obvious where to initalize the phases (q) from
# So let's run a pre-fit first, wherXe we only allow the phases to vary
# Trying running this a few times and make sure the same solution is coming out

# Define model M0 fix = 4-planet model using K & P parameters fixed to the Vogt+ reported values
def M0fix(t, qb, qc, qd, qe):
    return Kb_sol*np.sin(2.0*np.pi*t/Pb_sol+qb) \
           + Kc_sol*np.sin(2.0*np.pi*t/Pc_sol+qc) \
           + Kd_sol*np.sin(2.0*np.pi*t/Pd_sol+qd) \
           + Ke_sol*np.sin(2.0*np.pi*t/Pe_sol+qe)

# Define our parameters bounds
param_bounds = ( [-2.0*np.pi, \
                  -2.0*np.pi, \
                  -2.0*np.pi, \
                  -2.0*np.pi],\
                 [2.0*np.pi, \
                  2.0*np.pi, \
                  2.0*np.pi, \
                  2.0*np.pi] )

# Give an intial guess to help the fitting routine
initial_guess = [ np.random.uniform(-np.pi,np.pi),\
                  np.random.uniform(-np.pi,np.pi),\
                  np.random.uniform(-np.pi,np.pi),\
                  np.random.uniform(-np.pi,np.pi)]

# Get the best fitting parameters
M0fix_best, M0fix_cov = curve_fit(M0fix,data[:,0],data[:,1],sigma=data[:,2],p0=initial_guess,bounds=param_bounds)

# Get the chi2
chi2 = 0.0
for i in range(len(data)):
    chi2 = chi2 + ( ( data[i,1] - M0fix( data[i,0],\
                                      M0fix_best[0],\
                                      M0fix_best[1],\
                                      M0fix_best[2],\
                                      M0fix_best[3]) ) / data[i,2])**2

# Print output
print chi2,len(data)

395.796125728 241


In [42]:
# OK, I'm satisfied the above chi2 looks reasonable, so set the phase solutions as initial guesses
qb_sol = M0fix_best[0]
qc_sol = M0fix_best[1]
qd_sol = M0fix_best[2]
qe_sol = M0fix_best[3]

In [45]:
# Now we're ready to run the full 4-planet fit

# Define model M0 = 4-planet model
def M0(t, Kb, Pb, qb, Kc, Pc, qc, Kd, Pd, qd, Ke, Pe, qe):
    return Kb*np.sin(2.0*np.pi*t/Pb+qb) \
           + Kc*np.sin(2.0*np.pi*t/Pc+qc) \
           + Kd*np.sin(2.0*np.pi*t/Pd+qd) \
           + Ke*np.sin(2.0*np.pi*t/Pe+qe)

# Define our parameters bounds
param_bounds = ( [0.0,  Pb_sol-3.0*Pb_del, -2.0*np.pi, \
                  0.0,  Pc_sol-3.0*Pc_del, -2.0*np.pi, \
                  0.0,  Pd_sol-3.0*Pd_del, -2.0*np.pi, \
                  0.0,  Pe_sol-3.0*Pe_del, -2.0*np.pi],\
                 [20.0, Pb_sol+3.0*Pb_del,  2.0*np.pi, \
                  20.0, Pc_sol+3.0*Pc_del,  2.0*np.pi, \
                  20.0, Pd_sol+3.0*Pd_del,  2.0*np.pi, \
                  20.0, Pe_sol+3.0*Pe_del,  2.0*np.pi] )

# Give an intial guess to help the fitting routine
initial_guess = [ Kb_sol, Pb_sol, qb_sol,\
                  Kc_sol, Pc_sol, qc_sol,\
                  Kd_sol, Pd_sol, qd_sol,\
                  Ke_sol, Pe_sol, qe_sol]

# Get the best fitting parameters
M0_best, M0_cov = curve_fit(M0,data[:,0],data[:,1],sigma=data[:,2],p0=initial_guess,bounds=param_bounds)

# Get the chi2
chi2 = 0.0
for i in range(len(data)):
    chi2 = chi2 + ( ( data[i,1] - M0( data[i,0],\
                                      M0_best[0],M0_best[1],M0_best[2],\
                                      M0_best[3],M0_best[4],M0_best[5],\
                                      M0_best[6],M0_best[7],M0_best[8],\
                                      M0_best[9],M0_best[10],M0_best[11]) ) / data[i,2])**2
# Get the loglike
loglike = -0.5*len(data)*np.log(2.0*np.pi) - np.sum( np.log(data[:,2]) ) - 0.5*chi2

# Print output
print chi2,loglike

390.394636635 -567.779618203
